In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [ ]:
spark = SparkSession.builder.appName("spark_sql_api").master("local[*]").getOrCreate()

In [4]:
df = (
    spark
    .read
    .option(key="delimiter", value=";")
    .option(key="header", value="true")
    .option(key="inferSchema", value="true")
    .option(key="encoding", value="ISO-8859-1")
    .csv("./data/precos-gasolina-etanol-3.csv")
)

In [5]:
df.printSchema()

root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



In [6]:
df_precos = df.select(
    'Estado - Sigla',
    'Produto',
    'Valor de Compra',
    'Valor de Venda',
    'Unidade de Medida'
)

In [7]:
df_precos.show(5)

+--------------+------------------+---------------+--------------+-----------------+
|Estado - Sigla|           Produto|Valor de Compra|Valor de Venda|Unidade de Medida|
+--------------+------------------+---------------+--------------+-----------------+
|            PB|          GASOLINA|           NULL|          5,59|       R$ / litro|
|            PB|GASOLINA ADITIVADA|           NULL|          5,63|       R$ / litro|
|            PB|            ETANOL|           NULL|          3,99|       R$ / litro|
|            PB|          GASOLINA|           NULL|          5,59|       R$ / litro|
|            PB|GASOLINA ADITIVADA|           NULL|          5,79|       R$ / litro|
+--------------+------------------+---------------+--------------+-----------------+
only showing top 5 rows



In [8]:
# verifying if field "Valor de Compra" is always null
df_precos.where(
    F.col('Valor de Compra').isNotNull()
).show(5)

+--------------+-------+---------------+--------------+-----------------+
|Estado - Sigla|Produto|Valor de Compra|Valor de Venda|Unidade de Medida|
+--------------+-------+---------------+--------------+-----------------+
+--------------+-------+---------------+--------------+-----------------+



In [9]:
# removing "Valor de Compra" since it is always null and casting "Valor de Compra"
df_precos = df.select(
    'Estado - Sigla',
    'Produto',
    'Valor de Venda',
    'Unidade de Medida'
).withColumn(
    "Valor de Venda",
    F.regexp_replace(string="Valor de Venda", pattern=",", replacement=".").cast("float")
)

In [10]:
df_precos_analise = df_precos.groupBy(
    F.col('Estado - Sigla'),
    F.col('Produto'),
    F.col('Unidade de Medida')
).agg(
    F.min(F.col('Valor de Venda')).alias("menor_valor"),
    F.max(F.col('Valor de Venda')).alias("maior_valor")
).withColumn(
    "diferenca",
    F.col("maior_valor") - F.col("menor_valor")
).orderBy("diferenca", ascending=False)

In [19]:
# showing the states with the biggest price differences where fuel is gasoline ("GASOLINA")
df_precos_analise.where(df_precos_analise.Produto == "GASOLINA").show()

+--------------+--------+-----------------+-----------+-----------+----------+
|Estado - Sigla| Produto|Unidade de Medida|menor_valor|maior_valor| diferenca|
+--------------+--------+-----------------+-----------+-----------+----------+
|            SP|GASOLINA|       R$ / litro|       4.59|       7.97| 3.3799996|
|            RJ|GASOLINA|       R$ / litro|       4.99|       7.49|       2.5|
|            AM|GASOLINA|       R$ / litro|       5.29|        7.7| 2.4099998|
|            RS|GASOLINA|       R$ / litro|       5.09|       6.96| 1.8699999|
|            PA|GASOLINA|       R$ / litro|       5.22|       6.99|      1.77|
|            BA|GASOLINA|       R$ / litro|       5.19|       6.91| 1.7199998|
|            PR|GASOLINA|       R$ / litro|       5.14|       6.79| 1.6500001|
|            PE|GASOLINA|       R$ / litro|       5.24|       6.89| 1.6500001|
|            MS|GASOLINA|       R$ / litro|       5.25|       6.84| 1.5900002|
|            ES|GASOLINA|       R$ / litro|       5.

In [20]:
# Showing the states with the biggest price differences where fuel is ethanol ("ETANOL")
df_precos_analise.where(df_precos_analise.Produto == "ETANOL").show()

+--------------+-------+-----------------+-----------+-----------+----------+
|Estado - Sigla|Produto|Unidade de Medida|menor_valor|maior_valor| diferenca|
+--------------+-------+-----------------+-----------+-----------+----------+
|            SP| ETANOL|       R$ / litro|       2.69|       5.69|       3.0|
|            PA| ETANOL|       R$ / litro|       3.45|        5.9|      2.45|
|            RS| ETANOL|       R$ / litro|       3.65|       5.99| 2.3399997|
|            PE| ETANOL|       R$ / litro|       3.39|       5.59|       2.2|
|            CE| ETANOL|       R$ / litro|       3.94|       5.95| 2.0099998|
|            AL| ETANOL|       R$ / litro|       3.19|       5.12| 1.9299998|
|            RJ| ETANOL|       R$ / litro|       3.45|       5.37| 1.9199998|
|            SC| ETANOL|       R$ / litro|       3.69|       5.59| 1.9000001|
|            ES| ETANOL|       R$ / litro|       3.59|       5.29|       1.7|
|            RN| ETANOL|       R$ / litro|       3.89|       5.3